In [1]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [13]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_swta_t')),
    'H-UNet-URPC-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_urpc_swta_t')),
    'H-UNet-CCT-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_cct_swta_t')),
}

In [14]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [15]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-CCT-SWTA-T,0,1,100,38.71,24.00
1,H-UNet-CCT-SWTA-T,0,5,100,39.73,24.79
2,H-UNet-CCT-SWTA-T,0,10,100,40.49,25.38
3,H-UNet-CCT-SWTA-T,0,20,100,41.47,26.16
4,H-UNet-CCT-SWTA-T,0,50,100,41.68,26.33
5,H-UNet-CCT-SWTA-T,0,75,100,41.73,26.37
6,H-UNet-CCT-SWTA-T,0,100,100,41.75,26.38
7,H-UNet-SWTA-T,0,1,100,37.59,23.14
8,H-UNet-SWTA-T,0,5,100,38.85,24.11
9,H-UNet-SWTA-T,0,10,100,40.69,25.55


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                     Mean CI 0.9%    Mean CI 0.9%
Model              Inv Temp Regime                               
H-UNet-CCT-SWTA-T  1        100     38.71     NaN   24.00     NaN
                   5        100     39.73     NaN   24.79     NaN
                   10       100     40.49     NaN   25.38     NaN
                   20       100     41.47     NaN   26.16     NaN
                   50       100     41.68     NaN   26.33     NaN
                   75       100     41.73     NaN   26.37     NaN
                   100      100     41.75     NaN   26.38     NaN
H-UNet-SWTA-T      1        100     37.59     NaN   23.14     NaN
                   5        100     38.85     NaN   24.11     NaN
                   10       100     40.69     NaN   25.55     NaN
                   20       100     41.58     NaN   26.25     NaN
                   50       100     41.81     NaN   26.43     NaN
                   75       100     41.85     NaN   26.46     NaN
                   100      100     41.87     NaN   26.48     NaN
H-UNet-URPC-SWTA-T 1        100     61.08     NaN   43.97     NaN
                   5        100     61.66     NaN   44.57     NaN
                   10       100     61.22     NaN   44.12     NaN
                   20       100     60.84     NaN   43.72     NaN
                   50       100     60.74     NaN   43.61     NaN
                   75       100     60.73     NaN   43.60     NaN
                   100      100     60.72     NaN   43.60     NaN

In [10]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_unet_swta_t')),
    #'H-EM-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_em_unet_swta_t')),
    #'H-UAMT-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_uamt_unet_swta_t')),
    #'H-CPS-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_cps_unet_swta_t')),
    #'H-URPC-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_urpc_unet_swta_t')),
    'H-CCT-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_cct_unet_swta_t')),
}

In [11]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [12]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-CCT-SWTA-T,0,1,1,68.79,52.43
1,H-CCT-SWTA-T,0,1,2,73.13,57.64
2,H-CCT-SWTA-T,0,1,5,76.56,62.02
3,H-CCT-SWTA-T,0,1,10,81.27,68.44
4,H-CCT-SWTA-T,0,1,20,83.40,71.52
...,...,...,...,...,...,...
345,H-CCT-SWTA-T,9,100,1,70.73,54.72
346,H-CCT-SWTA-T,9,100,2,67.77,51.25
347,H-CCT-SWTA-T,9,100,5,76.27,61.63
348,H-CCT-SWTA-T,9,100,10,79.30,65.70


Dice           Jaccard          
                                Mean   CI 0.9%    Mean   CI 0.9%
Model        Inv Temp Regime                                    
H-CCT-SWTA-T 1        1       70.320  1.087251  54.256  1.313413
                      2       71.666  1.542682  55.910  1.888512
                      5       76.915  1.061672  62.523  1.387496
                      10      81.342  0.884548  68.578  1.260510
                      20      84.572  0.754374  73.288  1.134570
             5        1       70.257  1.180405  54.190  1.419175
                      2       71.593  1.605548  55.829  1.954311
                      5       77.085  1.001347  62.744  1.325994
                      10      80.480  0.847438  67.359  1.187165
                      20      84.111  0.786973  72.602  1.158121
             10       1       70.474  1.102858  54.443  1.330479
                      2       71.117  1.503408  55.244  1.823451
                      5       76.786  1.313096  62.374  1.742787
                      10      81.294  0.933597  68.513  1.322568
                      20      84.703  0.677141  73.485  1.015198
             20       1       70.417  1.171467  54.376  1.418396
                      2       70.779  1.430102  54.828  1.727097
                      5       77.028  1.115051  62.675  1.494573
                      10      80.936  0.968634  68.011  1.372786
                      20      84.758  0.939730  73.579  1.413199
             50       1       70.572  1.056763  54.556  1.278207
                      2       70.996  1.621768  55.108  1.968261
                      5       77.190  0.982993  62.882  1.305549
                      10      80.857  1.032904  67.900  1.448435
                      20      84.780  0.527523  73.590  0.792032
             75       1       70.533  1.182984  54.518  1.442237
                      2       70.592  1.270954  54.593  1.541542
                      5       76.146  1.003864  61.512  1.326669
                      10      81.182  1.216830  68.372  1.745454
                      20      84.319  0.718063  72.906  1.062365
             100      1       70.418  1.160211  54.380  1.399730
                      2       70.866  1.393533  54.932  1.694960
                      5       77.116  1.219469  62.798  1.615277
                      10      80.934  1.202182  68.024  1.692598
                      20      84.774  0.644517  73.585  0.978790

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [4]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'UNet': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('unet')),
    'EM': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('em_unet')),
    'UAMT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('uamt_unet')),
    'CPS': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cps_unet')),
    'URPC': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('urpc_unet')),
    'CCT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cct_unet')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,CCT,0,1,1,67.77,51.25
1,CCT,0,1,2,70.38,54.29
2,CCT,0,1,5,74.36,59.19
3,CCT,0,1,10,81.15,68.27
4,CCT,0,1,20,82.44,70.12
...,...,...,...,...,...,...
295,URPC,9,1,1,69.42,53.16
296,URPC,9,1,2,67.94,51.44
297,URPC,9,1,5,73.03,57.52
298,URPC,9,1,10,78.28,64.31


Dice           Jaccard          
                         Mean   CI 0.9%    Mean   CI 0.9%
Model Inv Temp Regime                                    
CCT   1        1       69.508  1.127252  53.300  1.352967
               2       70.437  1.068993  54.395  1.278210
               5       76.722  1.187579  62.279  1.559367
               10      81.351  0.915017  68.590  1.283394
               20      84.936  0.875354  73.843  1.318532
CPS   1        1       70.001  0.916001  53.869  1.093109
               2       71.559  1.411084  55.767  1.718967
               5       76.543  1.090913  62.039  1.434285
               10      81.421  0.792441  68.683  1.116813
               20      84.449  0.764957  73.105  1.156714
EM    1        1       69.550  1.278488  53.360  1.541775
               2       70.791  1.531235  54.852  1.857711
               5       75.739  1.073473  60.984  1.389841
               10      78.292  0.821111  64.348  1.102924
               20      81.486  0.864500  68.783  1.229583
UAMT  1        1       69.654  1.193144  53.476  1.433769
               2       70.251  1.460897  54.201  1.766214
               5       75.576  0.951420  60.767  1.242891
               10      79.481  0.746279  65.965  1.013731
               20      83.269  0.741795  71.355  1.087672
UNet  1        1       68.469  0.663928  52.065  0.784732
               2       70.532  1.267591  54.521  1.523118
               5       76.499  1.264902  61.991  1.668055
               10      80.832  1.090293  67.869  1.537295
               20      84.509  0.485512  73.179  0.730902
URPC  1        1       68.618  0.563173  52.235  0.656471
               2       69.324  1.427276  53.104  1.724666
               5       74.747  1.191352  59.719  1.526734
               10      79.113  1.529320  65.520  2.077778
               20      82.635  1.909080  70.537  2.622215